In [1]:
import os
import sys
sys.path.append(os.path.join('..', '..'))

import datetime
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.models import Panel, Tabs, Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, ColumnDataSource, LabelSet, PointDrawTool, WheelZoomTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.models.widgets import Div
from bokeh.plotting import figure
from bokeh.layouts import layout
from bokeh.models.tools import HoverTool
from bokeh.palettes import Spectral4, Spectral3, Category20
import networkx as nx

# custom libraries
from mikesnowflake.analysis.snowFlakeAnalysis import SnowFlakeAnalysis
from mikesnowflake.access.colorAccess import ColorAccess

output_notebook(hide_banner=True)

In [2]:
user = ''
password = ''

In [3]:
START_DATE = datetime.datetime(2018, 12, 1)
END_DATE = datetime.datetime(2020, 1, 16)
SFA = SnowFlakeAnalysis(START_DATE, END_DATE, user, password, excludeEtl=False, verbose=False)

In [4]:
tableNames = ['ADAPTIVE_INSIGHTS_HOURLY_FACT', 'ADAPTIVE_INSIGHTS_DAILY_FACT']

# collect queryType history for visualization analysis for each table in df
qtHistory = {}  # collector of query type history keyed by table name
for tableName in tableNames:
    qtHistory[tableName] = SFA.getQueryTypeHistory(tableName)

# show query type history for certain tables in question
p = {}  # collector of bokeh charts keyed by table name
for tableName in tableNames:
    p[tableName] = figure(width=1900, height=800, x_axis_type='datetime')
    for c in qtHistory[tableName].columns:
        p[tableName].line(qtHistory[tableName].index.tolist(), qtHistory[tableName][c], line_width=2, name=c, legend_label=c, line_color=SFA.queryTypeColors[c])
    p[tableName].title.text = 'Daily SQL Command Count History for %s' % tableName
    p[tableName].title.align = 'center'
    hover = HoverTool(tooltips=[('Date', '@x{%F}'),
                                ("Query Type", "$name"),
                                ("Hits", "@y{0,0}")],
                      formatters={'x': 'datetime'})
    p[tableName].add_tools(hover)
    p[tableName].legend.click_policy="hide"
tabSize = 5
subLists = [tableNames[i * tabSize:(i + 1) * tabSize] for i in range((len(tableNames) + tabSize - 1) // tabSize )]
tabs = []
for subList in subLists:
    tabs.append([Tabs(tabs=[Panel(child=p[tableName], title=tableName) for tableName in subList], background='whitesmoke')])
show(layout([Div(text="Daily SQL Command Count History </b>", style={'font-size': '150%'}, width=1500, height=25)],
            [tabs]))

In [5]:
ca = ColorAccess()
for tableName in tableNames:
    history = SFA.getQueryTextHistory(tableName)

    colors = dict(zip(history['user_name'].unique().tolist(), ca.getColors(len(history['user_name'].unique()))))
    userHistory = {}
    for queryType, out in history.groupby('query_type'):
        userHistory[queryType] = out.groupby(['query_date', 'user_name'])['query_id'].count().unstack().reindex(pd.date_range(START_DATE, END_DATE)).fillna(0)

    # collect queryType history by user for visualization analysis
    q = {}  # collector of bokeh charts keyed by table name
    for queryType, out in userHistory.items():
        q[queryType] = figure(width=1900, height=800, x_axis_type='datetime')
        for user in out.columns:
            q[queryType].line(out.index.tolist(), out[user], line_width=2, name=user, legend_label=user, line_color=colors[user])
        q[queryType].title.text = '%s History (%s)' % (tableName, queryType)
        q[queryType].title.align = 'center'
        hover = HoverTool(tooltips=[('Date', '@x{%F}'),
                                    ("User", "$name"),
                                    ("Hits", "@y{0,0}")],
                          formatters={'x': 'datetime'})
        q[queryType].add_tools(hover)
        q[queryType].legend.click_policy="hide"

    tabSize = 5
    queryTypes = [qt for qt in SFA.queryTypes['select'] if qt in history['query_type'].unique()]
    subLists = [queryTypes[i * tabSize:(i + 1) * tabSize] for i in range((len(queryTypes) + tabSize - 1) // tabSize )]
    tabs = []
    for subList in subLists:
        tabs.append([Tabs(tabs=[Panel(child=q[queryType], title=queryType) for queryType in subList], background='whitesmoke')])

    for tab in tabs:
        show(layout([Div(text="Select-like Command User History for %s</b>" % tableName, style={'font-size': '150%'}, width=1500, height=25)],
                    [tab]))

In [6]:
G = SFA.tableGraph
H = G.edge_subgraph([e for n in tableNames for e in G.edges(n)])
defaultColor = Category20[3][1]
defaultSize = '12px'
colorMap = {n: {'color': 'red'} if n in tableNames else {'color': defaultColor} for n in H.nodes()}  # highlight the node as red
title = 'Table Dependency Graph'
nx.set_node_attributes(H, colorMap)
    
plot = figure(title=title, x_range=(-1.1,1.1), y_range=(-1.1,1.1), height=1000, width=1500, tools="pan,box_zoom,reset")
plot.title.align = 'center'

graph_renderer = from_networkx(H, nx.spring_layout, scale=1, center=(0,0))
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color='color')
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=1, line_width=1)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=3)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=3)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

wheel_zoom = WheelZoomTool()
plot.add_tools(HoverTool(tooltips=None), TapTool(), wheel_zoom)
plot.toolbar.active_scroll=wheel_zoom
plot.renderers.append(graph_renderer)
pos = graph_renderer.layout_provider.graph_layout
x,y=zip(*pos.values())
source = ColumnDataSource({'x':x,'y':y, 'label': list(pos.keys())})
labels = LabelSet(x='x', y='y', text='label', source=source, text_font_size='11px')
plot.renderers.append(labels)
show(plot)

In [7]:
%%html
<style>
div.input {
    display:none;
}
</style>